# 0. Introduction and Schema Diagram

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

# 1. Overview of the Data

In [11]:
%%sql
select 
    e.first_name || " " || e.last_name as customer_name,
    e.country, 
    SUM(i.total) as total
        from employee as e
        inner join customer as c on c.support_rep_id = e.employee_id
        inner join invoice as i on i.invoice_id = c.customer_id

Done.


[('Steve Johnson', 'Canada', 464.30999999999995)]

In [24]:
%%sql
select 
    *
        from employee as e

Done.


[(1, 'Adams', 'Andrew', 'General Manager', None, '1962-02-18 00:00:00', '2016-08-14 00:00:00', '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com'),
 (2, 'Edwards', 'Nancy', 'Sales Manager', 1, '1958-12-08 00:00:00', '2016-05-01 00:00:00', '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com'),
 (3, 'Peacock', 'Jane', 'Sales Support Agent', 2, '1973-08-29 00:00:00', '2017-04-01 00:00:00', '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com'),
 (4, 'Park', 'Margaret', 'Sales Support Agent', 2, '1947-09-19 00:00:00', '2017-05-03 00:00:00', '683 10 Street SW', 'Calgary', 'AB', 'Canada', 'T2P 5G3', '+1 (403) 263-4423', '+1 (403) 263-4289', 'margaret@chinookcorp.com'),
 (5, 'Johnson', 'Steve', 'Sales Support Agent', 2, '1965-03-03 00:00:00', '2017-10-17 00:00:00', '7727B 41 Ave', 'Calgary', 'AB', 'Canada', 'T3B 1Y7', '1 (780) 836-9987', '1 (780) 836-9543', 'steve@chinookcorp.com'),
 (6, 'Mitchell', 'Michael', 'IT Manager', 1, '1973-07-01 00:00:00', '2016-10-17 00:00:00', '5827 Bowness Road NW', 'Calgary', 'AB', 'Canada', 'T3B 0C5', '+1 (403) 246-9887', '+1 (403) 246-9899', 'michael@chinookcorp.com'),
 (7, 'King', 'Robert', 'IT Staff', 6, '1970-05-29 00:00:00', '2017-01-02 00:00:00', '590 Columbia Boulevard West', 'Lethbridge', 'AB', 'Canada', 'T1K 5N8', '+1 (403) 456-9986', '+1 (403) 456-8485', 'robert@chinookcorp.com'),
 (8, 'Callahan', 'Laura', 'IT Staff', 6, '1968-01-09 00:00:00', '2017-03-04 00:00:00', '923 7 ST NW', 'Lethbridge', 'AB', 'Canada', 'T1H 1Y8', '+1 (403) 467-3351', '+1 (403) 467-8772', 'laura@chinookcorp.com')]

# 2. Selecting New Albums to Purchase

In [68]:
%%sql
with usa_tracks_sold as 
    (
    select il.*
        from invoice_line as il
        inner join invoice as i on i.invoice_id = il.invoice_id
        inner join customer as c on c.customer_id = i.customer_id
        where c.country = 'USA'
    )
    
select 
    g.name as genre,
    count(uts.invoice_line_id) as tracks_sold,
    cast(count(uts.invoice_line_id) as float)
    /
    (
    select count(*) from usa_tracks_sold
    ) as percentage
        from usa_tracks_sold as uts
        inner join track as t on t.track_id = uts.track_id
        inner join genre as g on g.genre_id = t.genre_id
            group by 1 order by 3 DESC
        
    
    

Done.


[('Rock', 561, 0.5337773549000951),
 ('Alternative & Punk', 130, 0.12369172216936251),
 ('Metal', 124, 0.11798287345385347),
 ('R&B/Soul', 53, 0.05042816365366318),
 ('Blues', 36, 0.03425309229305423),
 ('Alternative', 35, 0.03330161750713606),
 ('Latin', 22, 0.02093244529019981),
 ('Pop', 22, 0.02093244529019981),
 ('Hip Hop/Rap', 20, 0.019029495718363463),
 ('Jazz', 14, 0.013320647002854425),
 ('Easy Listening', 13, 0.012369172216936251),
 ('Reggae', 6, 0.005708848715509039),
 ('Electronica/Dance', 5, 0.004757373929590866),
 ('Classical', 4, 0.003805899143672693),
 ('Heavy Metal', 3, 0.0028544243577545195),
 ('Soundtrack', 2, 0.0019029495718363464),
 ('TV Shows', 1, 0.0009514747859181732)]

#### Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following artists:

##### - Red Tone (Punk)
##### - Slim Jim Bites (Blues)
##### - Meteor and the Girls (Pop)
#### It's worth keeping in mind that combined, these three genres only make up only 17% of total sales, so we should be on the lookout for artists and albums from the rock genre, which accounts for 53% of sales.

# 3. Analyzing Employee Sales Performance

#### Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

#### You might like to consider whether any extra columns from the employee table explain any variance you see, or whether the variance might instead be indicative of employee performance.


In [101]:
%%sql
with customer_support_rep_sales as 
    (select
        i.customer_id, 
         c.support_rep_id,
         sum(i.total) as total
            from customer as c 
            inner join invoice as i on i.customer_id = c.customer_id
            group by 1,2
    )
    
select 
    e.first_name || " " || e.last_name as employee_name, 
    e.hire_date,
    SUM(csrs.total) as total_sales
        from customer_support_rep_sales as csrs
        inner join employee as e on e.employee_id = csrs.support_rep_id
        group by 1 order by 3 desc;

Done.


[('Jane Peacock', '2017-04-01 00:00:00', 1731.5099999999998),
 ('Margaret Park', '2017-05-03 00:00:00', 1584.0000000000002),
 ('Steve Johnson', '2017-10-17 00:00:00', 1393.92)]

#### While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds to the differences in their hiring dates.



# 4. Analyzing Sales by Country

#### Your next task is to analyze the sales data for customers from each different country. You have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

#### In particular, you have been directed to calculate data, for each country, on the:

##### - total number of customers
##### - total value of sales
##### - average value of sales per customer
##### - average order value

#### Because there are a number of countries with only one customer, you should group these customers as "Other" in your analysis. You can use the following 'trick' to force the ordering of "Other" to last in your analysis.

#### If there is a particular value that you would like to force to the top or bottom of results, you can put what would normally be your most outer query in a subquery with a case statement that adds a numeric column, and then in the outer query sort by that column. Here's an example - let's start by creating a view so we're working with a manageable number of rows:

In [139]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

Done.


[('USA', 13, 1040.490000000008, 7.942671755725252, 80.03769230769292),
 ('Canada', 8, 535.5900000000034, 7.047236842105309, 66.94875000000043),
 ('Brazil', 5, 427.68000000000245, 7.011147540983647, 85.53600000000048),
 ('France', 5, 389.0700000000021, 7.781400000000042, 77.81400000000042),
 ('Germany', 4, 334.6200000000016, 8.161463414634186, 83.6550000000004),
 ('Czech Republic', 2, 273.24000000000103, 9.108000000000034, 136.62000000000052),
 ('United Kingdom', 3, 245.52000000000078, 8.768571428571457, 81.84000000000026),
 ('Portugal', 2, 185.13000000000022, 6.3837931034482835, 92.56500000000011),
 ('India', 2, 183.1500000000002, 8.72142857142858, 91.5750000000001),
 ('Other', 15, 1094.9400000000085, 7.448571428571486, 72.99600000000056)]

##### Based on the data, there may be opportunity in the following countries:

###### - Czech Republic
###### - United Kingdom
###### - India

##### It's worth remembering this because the amount of data from each of these countries is relatively low. As such, we should be cautious about spending too much money on new marketing campaigns because the sample size isn't large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

# 5. Albums vs. Individual Tracks


In [140]:
%%sql

WITH invoice_first_track AS (
  SELECT
    il.invoice_id AS invoice_id,
    MIN(il.track_id) AS first_track_id
  FROM
    invoice_line il
  GROUP BY
    1
)

-- Use a subquery to select the results of the invoice_first_track CTE and determine whether customers made album purchases
SELECT
  album_purchase,
  COUNT(invoice_id) AS number_of_invoices,
  CAST(COUNT(invoice_id) AS FLOAT) / (
    SELECT COUNT(*) FROM invoice
  ) AS percent
FROM
  (
    SELECT
      ifs.*,
      CASE
        -- Use the EXCEPT operator to compare the tracks in the first invoice with the tracks in subsequent invoices,
        -- and determine whether any tracks from the album were purchased in subsequent invoices.
        -- If the result of the EXCEPT is NULL, it means that all tracks from the album were purchased in subsequent invoices,
        -- and the customer made an album purchase.
        -- If the result of the EXCEPT is not NULL, it means that at least one track from the album was not purchased in subsequent invoices,
        -- and the customer did not make an album purchase.
        WHEN (
          SELECT
            t.track_id
          FROM
            track t
          WHERE
            t.album_id = (
              SELECT
                t2.album_id
              FROM
                track t2
              WHERE
                t2.track_id = ifs.first_track_id
            )
          EXCEPT
          SELECT
            il2.track_id
          FROM
            invoice_line il2
          WHERE
            il2.invoice_id = ifs.invoice_id
        ) IS NULL
        AND (
          SELECT
            il2.track_id
          FROM
            invoice_line il2
          WHERE
            il2.invoice_id = ifs.invoice_id
          EXCEPT
          SELECT
            t.track_id
          FROM
            track t
          WHERE
            t.album_id = (
              SELECT
                t2.album_id
              FROM
                track t2
              WHERE
                t2.track_id = ifs.first_track_id
            )
        ) IS NULL
        THEN "yes"
        ELSE "no"
      END AS album_purchase
    FROM
      invoice_first_track ifs
  ) subquery
-- Group by album_purchase to get the counts and percentages for each type of purchase
GROUP BY
  album_purchase;

Done.


[('no', 500, 0.8143322475570033), ('yes', 114, 0.18566775244299674)]

In [ ]:
###